In [ ]:
from netCDF4 import Dataset
import numpy as np
from wrf import(getvar,interplevel, to_np, latlon_coords, interpline, get_cartopy, cartopy_xlim, cartopy_ylim,ALL_TIMES, vertcross, smooth2d, CoordPair, GeoBounds)
import wrf
import glob
import xarray as xr
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def grid_xy(lat, lon, mlat,mlon):# The Lat/lon of paradise
    wp_lat = lat
    wp_lon = lon
    #Subtracting the station lat/lon from all the lat/lon points, them from there finding the minimum difference which gives the closest grid box index
    abslat = np.abs(mlat-wp_lat)
    abslon= np.abs(mlon-wp_lon)
    c = np.maximum(abslon,abslat)
    xx, yy = np.where(c == np.min(c))
    return(xx[0],yy[0]) 

In [ ]:
#Prepping Files For Import
#path =r'/export/home/mbrewer/Documents/campfire/stares/20181109' # use your path
allFiles = glob.glob('wrfout_d03_2018-11*')
print ('you have %s WRF files to merge' %(len(allFiles)))


for k,i in enumerate(allFiles,0): #pull time from import dictionaries and append to list
    ds = Dataset(i)
    d = xr.open_dataset(i)

    time = getvar(ds, "times",timeidx=ALL_TIMES)
    rh_2 = getvar(ds, 'rh2', timeidx = ALL_TIMES)
    lats, lons = latlon_coords(rh_2)
    u, v =getvar(ds, 'uvmet10', timeidx = ALL_TIMES)
    truck_lon = -121.57423166666667
    truck_lat = 39.69727166666667
    x, y = grid_xy(truck_lat, truck_lon, lats, lons)
    u_10 = u[:,x,y]
    v_10 = v[:,x,y]
    T2 = d.T2[:,x,y]
    rh2 = rh_2[:,x,y]
    sds = T2.to_dataset('T2')
    sds['u10'] = u_10
    sds['v10'] = v_10
    sds['rh2'] = rh2
    del sds['u10'].attrs['projection']
    del sds['v10'].attrs['projection']
    del sds['rh2'].attrs['projection']

    del sds['u10'].attrs['coordinates']
    del sds['v10'].attrs['coordinates']
    del sds['rh2'].attrs['coordinates']
    print('')
    print('processing WRF file at start time: %s' %(sds.Time[0].data))
    print('')
    if k ==0:
        sds.to_netcdf('timeseries0.nc')
    else:
        ts = xr.open_dataset('timeseries%s.nc'%(int(k-1))) 
        tss = xr.merge([ts, sds])
        tss.to_netcdf('timeseries%s.nc'%(k))
print('')        
print('timeseries%s is the final merged timeseries'%(len(allFiles)-1))